In [1]:
# general imports
import pandas as pd
import requests
import json
import ast

In [2]:
# load in files
keywords_df = pd.read_csv('data/keywords.csv')
movies_metadata_df = pd.read_csv('data/movies_metadata.csv')
bechdel_movies = pd.read_csv('data/bechdel_movies.csv')

/var/folders/lp/wvqcc0x13jjb99jsy0mqncqr0000gn/T/ipykernel_74026/4157074776.py:3: DtypeWarning: Columns (10) have mixed types. Specify dtype option on import or set low_memory=False.
  movies_metadata_df = pd.read_csv('data/movies_metadata.csv')


## Data Cleaning and Wrangling

Leave cells below commented, the cleaned and wrangled data has been uploaded into the `data` folder under `bechdel_movies.csv`, and has been stored in the the `bechdel_movies` variable.

In [3]:
# # General cleaning 

# # dropping unecessary columns from movies_metadata
# movies_metadata_df = movies_metadata_df.drop(columns = ['belongs_to_collection', 'homepage', 
#                                                         'poster_path', 'status', 'tagline', 
#                                                         'video', 'spoken_languages', 'vote_average', 
#                                                         'vote_count'])
# movies_metadata_df['id'] = pd.to_numeric(movies_metadata_df['id'] , downcast='integer', errors='coerce')

# # get average rating by 'movieId', drop unecessary columns, rename 'movieId'
# avg_rating = ratings_df.groupby('movieId').mean().reset_index()
# avg_rating = avg_rating.drop(columns = ['userId', 'timestamp'])
# avg_rating = avg_rating.rename(columns = {'movieId': 'id'})

# # joining movies_metadata_df and keywords_df together
# joined_df = pd.merge(movies_metadata_df, keywords_df, on='id', how='inner')
# # drop rows where 'id' or 'imdb_id' is NaN
# joined_df = joined_df.dropna(subset=['id', 'imdb_id'])

# # function to clean imdb_id of leading 'tt' for API calls
# def clean_imbd(imdb_ids):
#     return imdb_ids.replace('t', '')

# # apply function to imdb_id column 
# joined_df['imdb_id'] = joined_df['imdb_id'].apply(clean_imbd)

In [4]:
## Bechdel Test API Call - leave commented to avoid rerunning unecessarily 
## resulting .json can be found in data folder: 'data/bechdel_scores.json'

# url = "http://bechdeltest.com/api/v1/getAllMovies"
# response = requests.get(url)
# bechdel_scores = response.json()
# file_path = "bechdel_scores.json"

# with open(file_path, 'w') as json_file:
#     json.dump(bechdel_scores, json_file, indent=4)

# print(f"JSON data written to {file_path}")

In [5]:
# # create a pandas DataFrame with imdb_id and respective bechdel_score
# imdb_bechdel = {}
# for movie in bechdel_scores:
#     imdb_bechdel[movie['imdbid']] = movie['rating']
# bechdel_df = pd.DataFrame()
# bechdel_df['imdb_id'] = imdb_bechdel.keys()
# bechdel_df['bechdel_score'] = imdb_bechdel.values()

# # merge bechdel with movie dataset
# bechdel_movies = pd.merge(joined_df, bechdel_df, on='imdb_id', how='inner')

# extract names from columns with a list of dictionaries - returns a list with names
# def get_names(cell):
#     if pd.isna(cell):
#         return []
#     if isinstance(cell, str):
#         try:
#             cell = ast.literal_eval(cell)
#         except (ValueError, SyntaxError):
#             return []
#     names = []
#     for d in cell:
#         if isinstance(d, dict) and 'name' in d:
#             names.append(d['name'])
#     return names


# columns_to_clean = ['genres', 'production_companies', 'production_countries', 'keywords']
# for column in columns_to_clean:
#     bechdel_movies[column] = bechdel_movies[column].apply(get_names)

## Write DataFrame to .csv file - Leave commented
## resulting .csv can be found in data folder: 'data/bechdel_movies.csv'

#bechdel_movies.to_csv('bechdel_movies.csv', index=False, sep=',', na_rep='', encoding='utf-8')

In [6]:
# convert release_date to datetime object
bechdel_movies['release_date'] = pd.to_datetime(bechdel_movies['release_date'])

In [7]:
bechdel_movies.head()

,adult,budget,genres,id,imdb_id,original_language,original_title,overview,popularity,production_companies,production_countries,release_date,revenue,runtime,title,keywords,bechdel_score
0,False,30000000,"['Animation', 'Comedy', 'Family']",862.0,114709,en,Toy Story,"Led by Woody, Andy's toys live happily in his ...",21.946943,['Pixar Animation Studios'],['United States of America'],1995-10-30,373554033.0,81.0,Toy Story,"['jealousy', 'toy', 'boy', 'friendship', 'frie...",1
1,False,65000000,"['Adventure', 'Fantasy', 'Family']",8844.0,113497,en,Jumanji,When siblings Judy and Peter discover an encha...,17.015539,"['TriStar Pictures', 'Teitler Film', 'Intersco...",['United States of America'],1995-12-15,262797249.0,104.0,Jumanji,"['board game', 'disappearance', ""based on chil...",3
2,False,0,"['Romance', 'Comedy']",15602.0,113228,en,Grumpier Old Men,A family wedding reignites the ancient feud be...,11.712900,"['Warner Bros.', 'Lancaster Gate']",['United States of America'],1995-12-22,0.0,101.0,Grumpier Old Men,"['fishing', 'best friend', 'duringcreditssting...",3
3,False,16000000,"['Comedy', 'Drama', 'Romance']",31357.0,114885,en,Waiting to Exhale,"Cheated on, mistreated and stepped on, the wom...",3.859495,['Twentieth Century Fox Film Corporation'],['United States of America'],1995-12-22,81452156.0,127.0,Waiting to Exhale,"['based on novel', 'interracial relationship',...",2
4,False,60000000,"['Action', 'Crime', 'Drama', 'Thriller']",949.0,113277,en,Heat,"Obsessive master thief, Neil McCauley leads a ...",17.924927,"['Regency Enterprises', 'Forward Pass', 'Warne...",['United States of America'],1995-12-15,187436818.0,170.0,Heat,"['robbery', 'detective', 'bank', 'obsession', ...",2


In [8]:
bechdel_movies.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8069 entries, 0 to 8068
Data columns (total 17 columns):
 #   Column                Non-Null Count  Dtype         
---  ------                --------------  -----         
 0   adult                 8069 non-null   bool          
 1   budget                8069 non-null   int64         
 2   genres                8069 non-null   object        
 3   id                    8069 non-null   float64       
 4   imdb_id               8069 non-null   int64         
 5   original_language     8068 non-null   object        
 6   original_title        8069 non-null   object        
 7   overview              8060 non-null   object        
 8   popularity            8069 non-null   float64       
 9   production_companies  8069 non-null   object        
 10  production_countries  8069 non-null   object        
 11  release_date          8068 non-null   datetime64[ns]
 12  revenue               8069 non-null   float64       
 13  runtime           

## Exploratory Data Analysis & Visualization